# Análisis de datos

Para generar las slides use:

In [30]:
%%bash
jupyter nbconvert ALL_inspire.ipynb --to slides --reveal-prefix "https://cdnjs.cloudflare.com/ajax/libs/reveal.js/3.1.0"

[NbConvertApp] Converting notebook ALL_inspire.ipynb to slides
[NbConvertApp] Writing 273514 bytes to ALL_inspire.slides.html


In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth',200)

## Load data (WARNING: 1.5 GB)
Download from [here](https://drive.google.com/file/d/1NtocdsxbTky5uLmKp3eDoGcQG7qsTEYv/view?usp=sharing) 

In [3]:
df_all=pd.read_json('hep_records.json',lines=True)

In [4]:
df_all.shape

(1271720, 10)

## Keep only entries with references

In [5]:
dfr=df_all[df_all.references.map(len)>0].reset_index(drop=True)

In [593]:
dfr=dfr.reset_index(drop=True)

In [594]:
dfr.shape

(896637, 10)

### make a query

In [96]:
q=input('Search for author, e.g: restrepo, d:  ')

rq=dfr[dfr.full_authors.str.lower().str.contains('restrepo, d')].reset_index(drop=True)
print('{} records found'.format(rq.shape[0])  )
rq

Search for author, e.g: restrepo, d:  restrepo,d
46 records found


,abstract,authors,citations,co-authors,creation_date,free_keywords,recid,references,standardized_keywords,title,full_authors
0,"In the simplest scheme for neutrino masses invoking a triplet of Higgs scalars there are two CP-even neutral Higgs bosons $H_i$ (i=1,2) and one massive pseudoscalar $A$. For some choices of parame...","[Diaz, M.A.]","[468209, 614714, 1387338, 594326, 1465858]","[Garcia-Jareno, M.A., Restrepo, D.A., Valle, J.W.F.]",1997-12,[],452755,"[452289, 156195, 297673, 154794, 338828, 153987, 448471, 154264, 165562, 426586, 415276]","[electron positron: annihilation, Z0: electroproduction, Z0: width, electroweak interaction: validity test, Higgs particle: triplet, Higgs particle: mass, neutrino: mass, Higgs particle: hadronic ...",Neutrino mass and missing momentum Higgs boson signals,"Diaz, M.A.\nGarcia-Jareno, M.A.\nRestrepo, D.A.\nValle, J.W.F.\n"
1,"We perform a careful study of the neutral scalar sector of a model which includes a singlet, a doublet, and a triplet scalar field under $SU(2)$. This model is motivated by neutrino physics, since...","[Diaz, Marco A.]","[871431, 1618442, 594326, 1343128, 1454236, 818590, 733727, 511268, 496198, 871848, 1405993, 817322, 565163, 685357, 538417, 614714, 1419067, 484678, 1387338, 1477837, 611151, 475730, 1484516, 149...","[Garcia-Jareno, M.A., Restrepo, Diego A., Valle, J.W.F.]",1998-03,"[14.80.Cp, 13.85.Qk, 11.30.Fs, 12.60.Fr, 14.60.Pq]",468209,"[341121, 12291, 12292, 384903, 338828, 354450, 153987, 154264, 448153, 21019, 337948, 357919, 156195, 219303, 31786, 415276, 361648, 165562, 341307, 450497, 297673, 38603, 36949, 295257, 355805, 3...","[electron positron: annihilation, Higgs particle: electroproduction, Higgs particle: associated production, Majoron, neutrino: mass generation, electroweak interaction: model, Higgs particle: mult...",Seesaw Majoron model of neutrino mass and novel signals in Higgs boson production at LEP,"Diaz, Marco A.\nGarcia-Jareno, M.A.\nRestrepo, Diego A.\nValle, J.W.F.\n"
2,We study unconventional decays of the top-quark and the top-squark in the framework of SUSY models with broken R-parity. The model under study is the MSSM with an additional bilinear term that bre...,"[de Campos, F.]","[500354, 1325035, 618376, 1228048, 497300, 525081, 579226, 705563, 1296678, 780075, 530873, 508352, 1266784, 503363, 555590, 1234255, 784720, 561745, 509817, 510559, 500832, 505185, 1229521, 56663...","[Diaz, M.A., Eboli, Oscar J.P., Magro, M.B., Navarro, L., Porod, W., Restrepo, D.A., Valle, J.W.F.]",1998-11,[],483425,"[15873, 396546, 474677, 444170, 393099, 452880, 446097, 453140, 451477, 295257, 466802, 451738, 15646, 480287, 422690, 425251, 419885, 419377, 466483, 419381, 427318, 466366, 421951, 477316, 42771...","[talk: Batavia 1998/11/19, electroweak interaction, supersymmetry, R parity: violation, top: decay modes, squark: top, squark: decay, neutrino/tau: mass, top: branching ratio, squark: branching ra...",R-parity violating decays of the top quark and the top squark at the Tevatron,"de Campos, F.\nDiaz, M.A.\nEboli, Oscar J.P.\nMagro, M.B.\nNavarro, L.\nPorod, W.\nRestrepo, D.A.\nValle, J.W.F.\n"
3,We present an outlook for possible discovery of supersymmetry with broken R-parity at Run II of the Tevatron. We first present a review of the literature and an update of the experimental bounds. ...,"[Allanach, B.]","[607232, 607233, 563718, 601608, 667151, 1410582, 1624599, 530456, 562201, 1256960, 509103, 611357, 733727, 1395748, 612390, 733617, 805935, 887860, 538165, 1091641, 568890, 610875, 555069, 777279...","[Baer, H., Banerjee, S., Berger, Edmond L., Chertok, M.B., Cheung, King-man, de Campos, F., Dedes, A., Diaz, M.A., Dreiner, Herbert K., Eboli, Oscar J.P., Gunion, John F., Harris, B.W., Hewett, J....",1999-03,[],501153,"[30227, 453140, 466455, 429080, 478234, 478236, 37978, 480287, 394274, 441899, 419885, 419377, 466483, 419381, 396346, 448572, 421951, 455239, 322640, 427604, 371286, 398425, 393306, 424543, 30316...","[talk: Batavia 199

###  Convert to WOS-like format

In [171]:
dfr['full_authors']=dfr['authors']+dfr['co-authors']

In [172]:
dfrw=dfr.copy()
dfrw.full_authors=dfrw.full_authors.str.join('\n').str.replace('.$','\n')
#convert integer list to string list: https://stackoverflow.com/a/3590175/2268280
dfrw['refs']=dfrw.references.map(lambda x: '\n'.join( map( str,x )   )).str.replace('.$','\n')

In [177]:
dfrw[['title','full_authors','refs']][:3]#.to_excel('inspire.xlsx',index=False)

,title,full_authors,refs
0,A target beam monitor using secondary electron emission,"Johnston, Lawrence II.\nMurray, Joseph J.\nGearhart, Roger A.\nHauser, Todd\nKlein, Paul R\n",4864\n
1,Neutron-Proton Scattering Below 20-MeV,"Noyes, H.Pierr\n",47202\n43619\n47204\n46245\n47400\n45692\n438702\n48175\n458288\n3218\n48052\n46838\n9144\n1476836\n46874\n47483\n46684\n47418\n4688\n
2,Coherent Production as a Means of Determining the Spin and Parity of Bosons,"Berman, S.M.\nDrell, S.D\n",147582\n


## Build the network of references

In [405]:
df.shape

(1000, 11)

In [416]:
#df.to_json('full_ralations.json')

In [521]:
df[:1]

,abstract,authors,citations,co-authors,creation_date,free_keywords,recid,references,standardized_keywords,title,list_rcid
0,The exploitation of unrenormalized canonical field equations and equal-time commutation relations is generally not fruitful because of the need for infinite renormalization. The asymptotically fre...,"[Brandt, Richard A.]",[18551],"[Ng, Wing-Chiu, Young, Kenneth]",1976-05,[],3560,"[87072, 1283, 86276, 1577, 95084, 54444, 99663, 58673, 67637, 93143, 79224]","[GAUGE FIELD THEORY: NONABELIAN, GAUGE FIELD THEORY: ASYMPTOTIC FREEDOM, FIELD EQUATIONS, GAUGE FIELD THEORY: COMMUTATION RELATIONS, GAUGE FIELD THEORY: OPERATOR PRODUCT, COUPLING CONSTANT, RENORM...",Canonical Methods in Nonabelian Gauge Theories,"[3560, 3560, 3560, 3560, 3560, 3560, 3560, 3560, 3560, 3560, 3560]"


In [515]:
def check_intersect1d(x,y):
    return np.intersect1d(x,y)

def check_not_ref(x,ref):
    return x not in ref



#DEBUG: ====
#Nmin=1;DEBUG=False
def check_connections( dfi,Nmin=5,DEBUG=False,external_ed=pd.DataFrame() ):
#if True:    
    ed=pd.DataFrame()


    if DEBUG: 
        #===log===
        f=open('kk.log','w')
        f.write('')
        f.close()
        #=======
    #DEBUG===========
    #i=5000
    #if True:
    for i in dfi.index:
        #check if edge already extits in some external ed DataFrame===
        #edge=dfi.loc[i,'recid']
        #if not edges_already_exists(edge,external_ed):
        
        #================    
        if DEBUG:
            #===log===
            f=open('kk.log','a')
            f.write('{}\n'.format(i))
            f.close()
            #=======
        print(i,end='\r')
        # Drop already analysed files
        df_not_i=dfi.drop( dfi.index[ range(0,i+1)  ] ) 
        # prepare edges DataFrame
        df_not_i=df_not_i.rename_axis({'recid':'target'}, axis = 'columns')
        #WARNING: limit the number of columns to free memory
        df_not_i=df_not_i[['target','references']]
        # Make the intersection of dfi.loc[i,'references'] with all the others one (see previous restricion)
        df_not_i['intersect']=df_not_i['references'].map(lambda x: check_intersect1d(x,dfi.loc[i,'references']))
        # INPORTANT PART Restrict the number of edges ============
        df_not_i=df_not_i[df_not_i['intersect'].map(len)>=Nmin]
        #==========================================================
        # Add source column. 
        df_not_i['source']=dfi.loc[i,'recid']
        # drop references columns
        ed=ed.append( df_not_i[['source','target','intersect']] ).reset_index(drop=True) 

    return ed

In [517]:
import time
s=time.time()

dfi=dfr.sample(1000).sort_values('recid').reset_index(drop=True)

ed=check_connections(dfi,Nmin=5,DEBUG=True)

print(time.time()-s)

7976.3978016376495


In [527]:
edf=ed.copy()

In [528]:
edf.shape

(3271, 3)

In [536]:
edf=edf[edf.intersect.map(len)>=5]
edf.shape

(124, 3)

## Find the colegios

In [537]:
colegios=pd.DataFrame()
j=0
col={}
col[j]=[]

In [538]:
#i=3
#if True:
for i in edf.source.unique():
    #print(i)
    if True:
        if not i in col[j]: # new colegio!
            cl=edf[edf.source==i]
            # TODO loop in target
            if cl.target.values[0] not in col[j]: # neither source not target in old colegio:
                j=i
                col[j] =list( np.unique( np.concatenate ( ( cl.source.values,cl.target.values )  ) )  )
            else:
                col[j].append(i)

In [539]:
cdf=pd.DataFrame( { 'source': list( col.keys()),'colegios':  list( col.values() )  } )

In [540]:
cdf=cdf[cdf.source>0].reset_index(drop=True)

In [541]:
cdff=cdf.merge(edf,on='source',how='left').fillna('')

In [545]:
colg=cdff[['source','target','colegios','intersect']].sort_values(
     ['source','target'],ascending=False).reset_index(drop=True)
colg[:1]

,source,target,colegios,intersect
0,1644265,1662910,"[1644265, 1653123, 1662910]","[593382, 659055, 760769, 779080, 796887, 919443, 1124337, 1124338, 1203133, 1293923, 1321709, 1468068, 1605397]"


In [555]:
colg['len_col']=colg.colegios.map(len)
colg.len_col.max()

12

In [561]:
colg[colg.len_col>5].drop_duplicates('source')#.shape

,source,target,colegios,intersect,len_col
18,1260794,1662910,"[1260794, 1260873, 1296453, 1644265, 1653123, 1662910]","[494953, 593382, 599622, 659055, 760769, 779080, 796887, 919443]",6
26,1230004,1596723,"[1230004, 1260794, 1260873, 1296453, 1596723, 1235726]","[581996, 589478, 712925, 877524, 912611]",6
48,823215,1662910,"[823215, 871874, 883930, 1230004, 1260873, 1296453, 1296927, 1434935, 1631169, 1644265, 1653123, 1662910]","[12291, 12292, 40440, 50073, 593382, 700668, 796887]",12


## Better efficiency
The current approach does not scale well

We follow here the usual approach to have a node file and build up an edges file.
The egdes file will be build in [pajek.ipynb](./pajek.ipynb)

In [562]:
def ones(x,y):
    return np.ones(len(x)).astype(int)*y

##  Pajek like Dataframe

### Save node file
JSON format keeps the python object structure

In [563]:
SAVE=False
if SAVE:
    df.to_json('nodes_small.json')

In [564]:
df['list_rcid']=df.references.combine( df.recid, func=ones)

In [565]:
pj=pd.DataFrame( { 'recid': np.concatenate (  tuple( df.list_rcid.values ) ),
                   'references': np.concatenate (  tuple( df.references.values ) )} )

In [566]:
pj=pj.sort_values(['references','recid'])

In [567]:
pj.shape

(23960, 2)

Remove entries with a single citation

In [568]:
pj['diff_refs_1']=np.concatenate (  (  pj.references.values[1:]-pj.references.values[:-1],
                            [   pj.references.values[-1]-pj.references.values[-2]   ] ) )
pj['diff_refs_2']=np.concatenate (  (  [   pj.references.values[-1]-pj.references.values[-2]   ],
                                         (pj.references.values[:-1]-pj.references.values[1:])
                                   ) )
pj['diff_refs']=pj.diff_refs_1*pj.diff_refs_2

In [569]:
pj=pj[pj.diff_refs==0].drop(['diff_refs_1','diff_refs_2','diff_refs'],axis='columns').reset_index(drop=True)

In [570]:
pj.shape

(4915, 2)

In [572]:
pj[:8]

,recid,references
0,15541,113
1,143630,113
2,385487,113
3,468709,113
4,1323465,113
5,398379,250
6,641951,250
7,231044,931


###  Automatize the process

In [586]:
%%writefile nodes_to_pajek.py
import numpy as np
import pandas as pd

def ones(x,y):
    return np.ones(len(x)).astype(int)*y

def nodes_to_pajek(df):
    df['list_rcid']=df.references.combine( df.recid, func=ones)
    pj=pd.DataFrame( { 'recid': np.concatenate (  tuple( df.list_rcid.values ) ),
                   'references': np.concatenate (  tuple( df.references.values ) )} )
    pj=pj.sort_values(['references','recid'])
    pj['diff_refs_1']=np.concatenate (  (  pj.references.values[1:]-pj.references.values[:-1],
                            [   pj.references.values[-1]-pj.references.values[-2]   ] ) )
    pj['diff_refs_2']=np.concatenate (  (  [   pj.references.values[-1]-pj.references.values[-2]   ],
                                         (pj.references.values[:-1]-pj.references.values[1:])
                                   ) )
    pj['diff_refs']=pj.diff_refs_1*pj.diff_refs_2
    
    pj=pj[pj.diff_refs==0].drop(['diff_refs_1','diff_refs_2','diff_refs'],
                        axis='columns').reset_index(drop=True)
    return  pj

Overwriting nodes_to_pajek.py


In [584]:
from nodes_to_pajek import *

In [ ]:
pj=nodes_to_pajek(df)

In [578]:
pj.shape

(4915, 2)

### We can now build the edges file!!!!
In [pajek.ipynb](./pajek.ipynb)

In [587]:
def nodes_to_pajek_tmp(df):
    df['list_rcid']=df.references.combine( df.recid, func=ones)
    pj=pd.DataFrame( { 'recid': np.concatenate (  tuple( df.list_rcid.values ) ),
                   'references': np.concatenate (  tuple( df.references.values ) )} )
    pj=pj.sort_values(['references','recid'])
    pj['diff_refs_1']=np.concatenate (  (  pj.references.values[1:]-pj.references.values[:-1],
                            [   pj.references.values[-1]-pj.references.values[-2]   ] ) )
    pj['diff_refs_2']=np.concatenate (  (  [   pj.references.values[-1]-pj.references.values[-2]   ],
                                         (pj.references.values[:-1]-pj.references.values[1:])
                                   ) )
    pj['diff_refs']=pj.diff_refs_1*pj.diff_refs_2
    
    pj=pj[pj.diff_refs==0].drop(['diff_refs_1','diff_refs_2','diff_refs'],
                        axis='columns').reset_index(drop=True)
    return  pj

In [595]:
pjt=nodes_to_pajek_tmp(dfr)

In [596]:
pjt.shape

(21733815, 2)

In [597]:
pjt[:3]

,recid,references
0,76215,1
1,81651,1
2,128250,1


In [ ]:
pjt.to_json('')